# Creating a Simple LLM App

##  Loading ENV Variables

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


## Loading OpenAI Key
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
## Loading Langchain Key
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

## Data Ingestion -- From Website

In [2]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/Developers/observability")
documents = loader.load()

In [13]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Developers/observability', 'title': 'Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='\n\n\n\n\nAdd observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsDevelopersAdd observability to your LLM applicationEvaluate your LLM applicationOptimize a classifierRAG EvaluationsBacktestingEvaluate an agentRunning SWE-bench with LangSmithHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsDevelopersAdd observability to your LLM applicationOn this pageAdd observability to your LLM applicationObservability is important for any software application, but especially so for LLM applications.\nLLMs are non

## Chunking

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 100)
documents = text_splitter.split_documents(documents)

In [15]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Developers/observability', 'title': 'Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Developers/observability', 'title': 'Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsDevelopersAdd observability to your LLM applicationEvaluate your LLM applicationOptimize a classifierRAG EvaluationsBacktestingEvaluate an agentR

## Embedding & Querying

In [16]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [18]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [19]:
query = " What is Tracing an LLM ?"
result = vectorstoredb.similarity_search(query)
result

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Developers/observability', 'title': 'Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content="LangSmith is built for tracing the entire LLM pipeline - so let's do that!"),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Developers/observability', 'title': 'Add observability to your LLM application | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'Observability is important for any software application, but especially so for LLM applications.', 'language': 'en'}, page_content='LLMs are non-deterministic by nature, meaning they can produce unexpected results.\nThis makes them trickier than normal to debug.Luckily, this is where LangSmith can help!\nLangSmith has LLM-native observability, allowing you to get meaningf

## Retrieval Chain & Document Chain

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")


prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

    """
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n    '))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1306b0070>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1306b1990>, root_client=<openai.OpenAI object at 0x1304580a0>, root_async_client=<openai.AsyncOpenAI object at 0x1306b00a0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [ ]:
document_chain.invoke(
    {
        "input":"he first thing you might want to trace is all your OpenAI calls. ",
        "context": " " ## Need Context 
    }
)

## Retrievers

In [23]:
vectorstore_retriever = vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain

complete_chain = create_retrieval_chain(vectorstore_retriever, document_chain)

In [24]:
complete_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11114caf0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1306b0070>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1306b1990>, root_cl

In [26]:
response = complete_chain.invoke({"input":"he first thing you might want to trace is all your OpenAI calls."})
response['answer']

'Based on the provided context, calling `rag("where did harrison work")` will produce a trace of just the entire pipeline, with the OpenAI call included as a child run.'